In [1]:
import pandas as pd
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import os
import xarray as xr

In [2]:

def isFilePresent( file_name):
    file_present = os.path.isfile(file_name)
    if file_present:
        print('Alreaday present : '+file_name)
    else:
        print('file absent : '+file_name)
    return file_present

In [3]:
def download_climate_data_xarray(date, latitude, longitude, area, day_night_ind, result_df):
    
    c = cdsapi.Client()
    
    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]
    
    area_str = '_'.join(str(num) for num in area)
    
    file_name = './historical_climate_data/'+year+'/weather-'+str(longitude)+'-'+str(latitude)+'-'+date+'.nc'
    if not isFilePresent(file_name):
        
        time = '12:00'
        if day_night_ind == "N":
            time = '22:00'
        c.retrieve('reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation', 'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4'
            ],
            'year': year,
            'month': month,
            'day': [str(day)],
            'time': [time],
            'area': area,
        },
        file_name)

    ds = xr.open_dataset(file_name)

    longitude = np.mean(ds['longitude'].values)
    latitude = np.mean(ds['latitude'].values)
    u10 = np.mean(ds['u10'].values)
    v10 = np.mean(ds['v10'].values)
    t2m = np.mean(ds['t2m'].values)
    stl1 = np.mean(ds['stl1'].values)
    stl2 = np.mean(ds['stl2'].values)
    stl3 = np.mean(ds['stl3'].values)
    stl4 = np.mean(ds['stl4'].values)
    slt = np.mean(ds['slt'].values)
    tp = np.mean(ds['tp'].values)
    swvl1 = np.mean(ds['swvl1'].values)
    swvl2 = np.mean(ds['swvl2'].values)
    swvl3 = np.mean(ds['swvl3'].values)
    swvl4 = np.mean(ds['swvl4'].values)

    new_row = {'longitude':longitude, 
           'latitude':latitude, 
           'date': date,       
           'daynight': day_night_ind,
           '10m_u_component_of_wind': u10,
           '10m_v_component_of_wind': v10,
           '2m_temperature': t2m,
           'soil_temperature_level_1': stl1,
           'soil_temperature_level_2': stl2,
           'soil_temperature_level_3': stl3,
           'soil_temperature_level_4': stl4,
           'soil_type': slt,
           'total_precipitation': tp,
           'volumetric_soil_water_layer_1': swvl1,
           'volumetric_soil_water_layer_2': swvl2,
           'volumetric_soil_water_layer_3': swvl3,
           'volumetric_soil_water_layer_4': swvl4}
    result_df.loc[len(result_df)] = new_row
    

In [4]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    #grouped_fire_data.apply(apply_on_each_group)

    result_df = pd.DataFrame({'longitude': pd.Series(dtype='float'),
                   'latitude': pd.Series(dtype='float'),
                   'date': pd.Series(dtype='str'),       
                   'daynight': pd.Series(dtype='str'),                   
                   '10m_u_component_of_wind': pd.Series(dtype='float'),
                   '10m_v_component_of_wind': pd.Series(dtype='float'),
                   '2m_temperature': pd.Series(dtype='float'),
                   'soil_temperature_level_1': pd.Series(dtype='float'),
                   'soil_temperature_level_2': pd.Series(dtype='float'),
                   'soil_temperature_level_3': pd.Series(dtype='float'),
                   'soil_temperature_level_4': pd.Series(dtype='float'),
                   'soil_type': pd.Series(dtype='float'),
                   'total_precipitation': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_1': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_2': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_3': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_4': pd.Series(dtype='float')})
    
    #print(grouped_fire_data)
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)
            download_climate_data_xarray(date_value, latitude, longitude, area, day_night_ind, result_df)
    
    result_filename = "./historical_climate_data/csv/"+file_name+".csv"
    result_df.to_csv(result_filename)
    return result_filename
    

In [5]:
#group_fire_data_xarray("canada_2023")

In [6]:
#group_fire_data_xarray("canada_2022")

In [7]:
import concurrent.futures


# Define parameters
parameters = ["canada_2022", "canada_2021", "canada_2020", "canada_2019", "canada_2018", "canada_2017", "canada_2016", "canada_2015", "canada_2014", "canada_2013", "canada_2012", "canada_2011", "canada_2010"] 

# Create a ThreadPoolExecutor with 5 worker threads
with concurrent.futures.ThreadPoolExecutor(max_workers=13) as executor:
    # Submit each function call with a parameter
    future_to_parameter = {executor.submit(group_fire_data_xarray, parameter): parameter for parameter in parameters}
    
    # Iterate through the completed futures
    for future in concurrent.futures.as_completed(future_to_parameter):
        parameter = future_to_parameter[future]
        try:
            # Get the result of the function call
            result = future.result()
            print(f"Parameter: {parameter}, Result: {result}")
        except Exception as e:
            print(f"Parameter: {parameter}, Exception: {e}")

file absent : ./historical_climate_data/2020/weather--115.849-55.696-2020-01-04.nc
Alreaday present : ./historical_climate_data/2022/weather--120.7944-55.339499999999994-2022-01-11.nc
file absent : ./historical_climate_data/2019/weather--115.17660000000001-56.17275-2019-01-01.nc
file absent : ./historical_climate_data/2018/weather--122.44236666666666-55.917966666666665-2018-01-01.nc
file absent : ./historical_climate_data/2021/weather--118.1776-54.53805-2021-01-01.nc
Alreaday present : ./historical_climate_data/2022/weather--115.60915-54.6095-2022-01-12.nc
Alreaday present : ./historical_climate_data/2022/weather--118.8242-54.606750000000005-2022-01-13.nc
Alreaday present : ./historical_climate_data/2022/weather--120.15755-54.888774999999995-2022-01-14.nc
Alreaday present : ./historical_climate_data/2022/weather--116.46225000000001-54.32155-2022-01-14.nc
Alreaday present : ./historical_climate_data/2022/weather--119.93253333333332-55.749766666666666-2022-01-15.nc
Alreaday present : ./h

2024-02-06 18:20:17,767 INFO Welcome to the CDS
2024-02-06 18:20:17,767 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-02-06 18:20:17,785 INFO Welcome to the CDS
2024-02-06 18:20:17,786 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-02-06 18:20:17,812 INFO Welcome to the CDS
2024-02-06 18:20:17,813 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-02-06 18:20:17,818 INFO Welcome to the CDS
2024-02-06 18:20:17,819 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Alreaday present : ./historical_climate_data/2022/weather--122.32462000000001-52.71694-2022-03-08.nc
Alreaday present : ./historical_climate_data/2022/weather--62.632600000000004-45.38074-2022-03-09.nc
Alreaday present : ./historical_climate_data/2022/weather--122.583125-55.561525-2022-03-14.nc
Alreaday present : ./historical_climate_data/2022/weather--123.54984999999999-52.34465-2022-03-16.nc
Alreaday present : ./historical_climate_data/2022/weather--114.63766666666668-52.599266666666665-2022-03-18.nc
Alreaday present : ./historical_climate_data/2022/weather--113.9434-52.7443-2022-03-22.nc
Alreaday present : ./historical_climate_data/2022/weather--115.46835-53.4358-2022-03-23.nc
Alreaday present : ./historical_climate_data/2022/weather--124.1773-53.2059-2022-03-24.nc
Alreaday present : ./historical_climate_data/2022/weather--120.3765-56.2836-2022-03-31.nc
Alreaday present : ./historical_climate_data/2022/weather--113.8588-52.1764-2022-04-09.nc
Alreaday present : ./historical_climate_d

2024-02-06 18:20:17,886 INFO Request is queued
2024-02-06 18:20:17,920 INFO Request is queued
2024-02-06 18:20:17,931 INFO Request is queued
2024-02-06 18:20:17,948 INFO Request is queued
2024-02-06 18:20:18,045 INFO Welcome to the CDS
2024-02-06 18:20:18,045 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Alreaday present : ./historical_climate_data/2022/weather--64.6518-58.553-2022-06-08.nc
Alreaday present : ./historical_climate_data/2022/weather--98.7755875-55.1026-2022-06-09.nc
Alreaday present : ./historical_climate_data/2022/weather--72.52460967741936-57.97976451612903-2022-06-09.nc
Alreaday present : ./historical_climate_data/2022/weather--104.63295-55.384550000000004-2022-06-10.nc
Alreaday present : ./historical_climate_data/2022/weather--99.22722-56.59102-2022-06-12.nc
Alreaday present : ./historical_climate_data/2022/weather--99.242425-56.581325-2022-06-13.nc
Alreaday present : ./historical_climate_data/2022/weather--103.6598-53.0285-2022-06-13.nc
Alreaday present : ./historical_climate_data/2022/weather--101.12070833333333-58.881499999999996-2022-06-14.nc
Alreaday present : ./historical_climate_data/2022/weather--102.6578-57.883849999999995-2022-06-14.nc
Alreaday present : ./historical_climate_data/2022/weather--109.49645757575757-59.330918181818184-2022-06-15.nc
Alreaday pre

2024-02-06 18:20:18,171 INFO Request is queued


KeyboardInterrupt: 